##Data Cleaning

In [94]:
import json
import pandas as pd
import numpy as np

In [3]:
with open("tmp/organizations_dict.json", "r") as fd:
    orgs = json.load(fd)

In [21]:
df=pd.DataFrame(orgs.values(),index=orgs.keys())

Copied this from HW5, to get adjectives and nouns from the short_description

In [ ]:
from sklearn.feature_extraction import text 
stopwords=text.ENGLISH_STOP_WORDS
import re
regex1=re.compile(r"\.{2,}")
regex2=re.compile(r"\-{2,}")
from pattern.en import parse
from pattern.en import pprint
from pattern.vector import stem, PORTER, LEMMA
punctuation = list('.,;:!?()[]{}`''\"@#$^&*+-|=~_')
# taken from HW5
def get_parts(thetext):
    thetext=re.sub(regex1, ' ', thetext)
    thetext=re.sub(regex2, ' ', thetext)
    nouns=[]
    descriptives=[]
    for i,sentence in enumerate(parse(thetext, tokenize=True, lemmata=True).split()):
        nouns.append([])
        descriptives.append([])
        for token in sentence:
            if len(token[4]) >0:
                if token[1] in ['JJ', 'JJR', 'JJS']:
                    if token[4] in stopwords or token[4][0] in punctuation or token[4][-1] in punctuation or len(token[4])==1:
                        continue
                    descriptives[i].append(token[4])
                elif token[1] in ['NN', 'NNS']:
                    if token[4] in stopwords or token[4][0] in punctuation or token[4][-1] in punctuation or len(token[4])==1:
                        continue
                    nouns[i].append(token[4])
    out=zip(nouns, descriptives)
    nouns2=[]
    descriptives2=[]
    for n,d in out:
        if len(n)!=0 and len(d)!=0:
            nouns2.append(n)
            descriptives2.append(d)
    return nouns2, descriptives2


In [82]:
# add column for description nouns and adjectives
df['text_nouns']=df.short_description.map(lambda x: None if x==None else get_parts(x)[0])
df['text_adjectives']=df.short_description.map(lambda x: None if x==None else get_parts(x)[1])

In [216]:
# add indicator columns for each category
catlist=df.categories.map(lambda x: None if x==None or isinstance(x,float) else [a['name'] for a in x]).tolist()
catset=set([item for sublist  in catlist if sublist!=None for item in sublist])
for category in catset:
    df["c_"+category+"_i"]=pd.Series(catlist, index=df.index).map(lambda x: category in x if x else False)

In [217]:
# add indicator columns for investors
investorslist=df.investors.map(lambda x: None if x==None or isinstance(x,float) else [a['permalink'] for a in x]).tolist()
investorset=set([item for sublist  in investorslist if sublist!=None for item in sublist])
for inv in investorset:
    df["i_"+inv+"_i"]=pd.Series(investorslist, index=df.index).map(lambda x: inv in x if x else False)

In [ ]:
# add indicator columns for cities (from offices)
citylist=df.offices.map(lambda x: None if not x or isinstance(x,float) else [a['city'] for a in x]).tolist()
cityset=set([item for sublist  in citylist if sublist!=None for item in sublist])
for city in cityset:
    if city != None:
        df["city_"+city+"_i"]=pd.Series(citylist, index=df.index).map(lambda x: city in x if x else False)

After adding indicator columns, we now get some more data from the fields that currently exist as dictionaries/lists

In [349]:
df['funding_dates']=df.funding_rounds.map(lambda x:[] if not x or isinstance(x,float) else [datetime.datetime(*time.strptime(a['announced_on'], '%Y-%m-%d')[:3]) for a in x])

In [374]:
df['funding_amounts']=df.funding_rounds.map(lambda x:[] if not x or isinstance(x,float) else [float(a['money_raised_usd']) if a['money_raised_usd'] != None else 0 for a in x])

In [371]:
df['num_funding_rounds']=df.funding_amounts.map(lambda x: len(x))

In [380]:
df['funding_mean']=df.funding_amounts.map(lambda x: np.mean(x))

In [389]:
df['went_ipo']=df.ipo.map(lambda x: True if x else False)

In [396]:
df['was_acquired']=df.acquired_by.map(lambda x: True if x else False)

In [411]:
df['founder_names']=df.founders.map(lambda x:[] if not x or isinstance(x,float) else [a['permalink']  for a in x])

In [485]:
df['founder_ages']= df.founders.map(lambda x:[] if not x or isinstance(x,float) else [(datetime.date.today()-datetime.date(*time.strptime(a['born_on'],'%Y-%m-%d')[:3])).days/365 for a in x if isinstance(a['born_on'],unicode)])

In [586]:
df

,acquired_by,acquisitions,board_members_and_advisors,categories,competitors,founders,funding_rounds,headquarters,investments,investors,...,city_Baltimore_i,city_Brentwood_i,city_Abingdon_i,city_New Haven_i,city_New Berlin_i,city_Amstelveen_i,city_Easley_i,city_Tourcoing_i,city_Oakland_i,city_Bra_i
24me,[],[],[],"[{u'name': u'Artificial Intelligence'}, {u'nam...",[],[{u'bio': u'Gilad is the Co-Founder and CEO of...,"[{u'announced_on': u'2014-05-02', u'funding_ty...",[],[],[{u'permalink': u'iangels'}],...,False,False,False,False,False,False,False,False,False,False
achieve-financial-services,[],[],[],"[{u'name': u'Finance'}, {u'name': u'P2P Money ...",[],[],"[{u'announced_on': u'2012-03-20', u'funding_ty...","[{u'city': u'Austin', u'country': u'United Sta...",[],[{u'permalink': u'escalate-capital-partners'}],...,False,False,False,False,False,False,False,False,False,False
4tiitoo,[],[],[],"[{u'name': u'Augmented Reality'}, {u'name': u'...",[],[{u'bio': u'Tore Meyer is Founder & CEO at 4ti...,"[{u'announced_on': u'2014-08-18', u'funding_ty...","[{u'city': u'Munich', u'country': u'Germany'},...",[],[{u'permalink': u'intel-capital'}],...,False,False,False,False,False,False,False,False,False,False
128-technology,[],[],[],"[{u'name': u'Technology'}, {u'name': u'Service...",[],[],"[{u'announced_on': u'2014-12-16', u'funding_ty...","[{u'city': u'Burlington', u'country': u'United...",[],[],...,False,False,False,False,False,False,False,False,False,False
-the-one-of-them-inc-,[],[],[],"[{u'name': u'Apps'}, {u'name': u'Mobile'}, {u'...",[],[],"[{u'announced_on': u'2014-01-30', u'funding_ty...",[],[],[],...,False,False,False,False,False,False,False,False,False,False
acquisition-glacier-ii,[],[],[],[{u'name': u'Health and Wellness'}],[],[],"[{u'announced_on': u'2015-07-21', u'funding_ty...","[{u'city': u'Westmount', u'country': u'Canada'}]",[],[],...,False,False,False,False,False,False,False,False,False,False
37coins,[],[],[],[{u'name': u'Bitcoin'}],[],"[{u'bio': u'Co-Founder of 37coins.', u'permali...","[{u'announced_on': u'2014-03-15', u'funding_ty...","[{u'city': u'Sunnyvale', u'country': u'United ...",[],"[{u'permalink': u'plug-and-play-ventures'}, {u...",...,False,False,False,False,False,False,False,False,False,False
abroad101,[],[],[],[{u'name': u'Education'}],[],[{u'bio': u'Adam Miller is the Founder and CEO...,"[{u'announced_on': u'2010-10-21', u'funding_ty...","[{u'city': u'Boston', u'country': u'United Sta...",[],[{u'permalink': u'masschallenge'}],...,False,False,False,False,False,False,False,False,False,False
3dr-laboratories,[],[],[],[{u'name': u'Biotechnology'}],[],"[{u'bio': None, u'permalink': u'robert-l-falk'...","[{u'announced_on': u'2012-08-23', u'funding_ty...","[{u'city': u'Louisville', u'country': u'United...",[],[{u'permalink': u'advantage-capital-partners'}],...,False,False,False,False,False,False,False,False,False,False
a-bit-lucky,[{u'announced_on': u'2012-09-17'}],[],[],"[{u'name': u'Games'}, {u'name': u'Social Games'}]",[],"[{u'bio': None, u'permalink': u'jordan-maynard...","[{u'announced_on': u'2010-03-15', u'funding_ty...","[{u'city': u'San Mateo', u'country': u'United ...",[],"[{u'permalink': u'sv-angel'}, {u'bio': u'David...",...,False,False,False,False,False,False,False,False,False,False
